# Prepare data

In [ ]:
from google.colab import files
files.upload()

In [2]:
!mkdir ~/.kaggle
!cp kaggle.json ~/.kaggle/
!chmod 600 ~/.kaggle/kaggle.json

In [3]:
# downloading dataset from kaggle
!kaggle competitions download -c dogs-vs-cats

 98% 796M/812M [00:04<00:00, 150MB/s]
100% 812M/812M [00:04<00:00, 176MB/s]


In [4]:
!unzip -qq dogs-vs-cats.zip
!unzip -qq train.zip

In [5]:
import os, shutil, pathlib

original_dir = pathlib.Path("train")
new_base_dir = pathlib.Path("dogs_vs_cats_small")

def make_subset(name:str, start:int, end:int):
    for category in ("dog", "cat"):
        dir = new_base_dir / name / category
        os.makedirs(dir)

        # create list of filename - first all dogs then all cats (category)
        fnames = [f"{category}.{i}.jpg" for i in range(start, end)]

        for fname in fnames:
            shutil.copyfile(src=original_dir / fname, dst=dir / fname)

In [6]:
make_subset("train", start=0, end=1_000)
make_subset("validation", start=1_000, end=1_500)
make_subset("test", start=1_500, end=2_500)

In [7]:
from tensorflow.keras.utils import image_dataset_from_directory

train_data = image_dataset_from_directory(
    new_base_dir / "train",
    image_size=(180, 180),
    batch_size=32
)

val_data = image_dataset_from_directory(
    new_base_dir / "validation",
    image_size=(180, 180),
    batch_size=32
)

test_data = image_dataset_from_directory(
    new_base_dir / "test",
    image_size=(180, 180),
    batch_size=32
)

Found 2000 files belonging to 2 classes.
Found 1000 files belonging to 2 classes.
Found 2000 files belonging to 2 classes.


# Build model and train classifier

In [8]:
from tensorflow.keras.applications import vgg16

conv_base = vgg16.VGG16(
    weights="imagenet",
    include_top=False
)
conv_base.trainable = False

58900480/58889256 [==============================] - 0s 0us/step


In [9]:
conv_base.summary()

Model: "vgg16"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input_1 (InputLayer)        [(None, None, None, 3)]   0         
                                                                 
 block1_conv1 (Conv2D)       (None, None, None, 64)    1792      
                                                                 
 block1_conv2 (Conv2D)       (None, None, None, 64)    36928     
                                                                 
 block1_pool (MaxPooling2D)  (None, None, None, 64)    0         
                                                                 
 block2_conv1 (Conv2D)       (None, None, None, 128)   73856     
                                                                 
 block2_conv2 (Conv2D)       (None, None, None, 128)   147584    
                                                                 
 block2_pool (MaxPooling2D)  (None, None, None, 128)   0     

In [10]:
# data augmentation
from tensorflow.keras.layers import RandomFlip, RandomRotation, RandomZoom
from tensorflow.keras.models import Sequential

data_aug = Sequential([
    RandomFlip("horizontal"),
    RandomRotation(0.1),
    RandomZoom(0.2)
])

In [11]:
# stacking model
from tensorflow.keras.models import Model
from tensorflow.keras.layers import Input, Dense, Flatten, Dropout

inputs = Input(shape=(180, 180, 3))
x = data_aug(inputs) # data augementation
x = vgg16.preprocess_input(x) # vgg16 value scaling
x = conv_base(x) # vgg16 features
x = Flatten()(x)
x = Dense(256, activation="relu")(x)
x = Dropout(0.5)(x)
outputs = Dense(1, activation="sigmoid")(x)

model = Model(inputs=inputs, outputs=outputs)

model.compile(optimizer="rmsprop",
              loss="binary_crossentropy",
              metrics=["accuracy"])

In [12]:
model.fit(train_data,
          epochs=50,
          validation_data=val_data
)

Epoch 1/50
63/63 [==============================] - 28s 198ms/step - loss: 4.0034 - accuracy: 0.8920 - val_loss: 0.5972 - val_accuracy: 0.9670
Epoch 2/50
63/63 [==============================] - 11s 167ms/step - loss: 1.0258 - accuracy: 0.9330 - val_loss: 0.9266 - val_accuracy: 0.9520
Epoch 3/50
63/63 [==============================] - 11s 168ms/step - loss: 0.6223 - accuracy: 0.9455 - val_loss: 0.2447 - val_accuracy: 0.9780
Epoch 4/50
63/63 [==============================] - 11s 168ms/step - loss: 0.5371 - accuracy: 0.9540 - val_loss: 0.2929 - val_accuracy: 0.9770
Epoch 5/50
63/63 [==============================] - 11s 168ms/step - loss: 0.3619 - accuracy: 0.9710 - val_loss: 0.3214 - val_accuracy: 0.9780
Epoch 6/50
63/63 [==============================] - 11s 169ms/step - loss: 0.4853 - accuracy: 0.9600 - val_loss: 0.4811 - val_accuracy: 0.9700
Epoch 7/50
63/63 [==============================] - 11s 170ms/step - loss: 0.3475 - accuracy: 0.9640 - val_loss: 1.3309 - val_accuracy: 0.9470

# Fine-tune the model

In [13]:
# freeze all layers until the foruth from last
conv_base.trainable = True
for layer in conv_base.layers[:-4]:
    layer.trainable = False

In [14]:
from tensorflow.keras.callbacks import ModelCheckpoint

MODEL_PATH = "fine-tuned-model.keras"

callbacks = [
    ModelCheckpoint(
        filepath=MODEL_PATH,
        save_best_only=True,
        monitor="val_loss"
    )
]

In [17]:
from tensorflow.keras.optimizers import RMSprop

model.compile(optimizer=RMSprop(learning_rate=1e-5),
              loss="binary_crossentropy",
              metrics=["accuracy"])

In [19]:
model.fit(train_data, 
          epochs=30,
          validation_data=val_data,
          callbacks=callbacks)

Epoch 1/30
63/63 [==============================] - 13s 196ms/step - loss: 0.0305 - accuracy: 0.9960 - val_loss: 0.3045 - val_accuracy: 0.9860
Epoch 2/30
63/63 [==============================] - 12s 189ms/step - loss: 0.0783 - accuracy: 0.9960 - val_loss: 0.3102 - val_accuracy: 0.9810
Epoch 3/30
63/63 [==============================] - 12s 190ms/step - loss: 0.0520 - accuracy: 0.9950 - val_loss: 0.4198 - val_accuracy: 0.9800
Epoch 4/30
63/63 [==============================] - 12s 194ms/step - loss: 0.0563 - accuracy: 0.9945 - val_loss: 0.2815 - val_accuracy: 0.9830
Epoch 5/30
63/63 [==============================] - 12s 190ms/step - loss: 0.0685 - accuracy: 0.9945 - val_loss: 0.3106 - val_accuracy: 0.9820
Epoch 6/30
63/63 [==============================] - 13s 197ms/step - loss: 0.0425 - accuracy: 0.9970 - val_loss: 0.3847 - val_accuracy: 0.9840
Epoch 7/30
63/63 [==============================] - 12s 191ms/step - loss: 0.0602 - accuracy: 0.9970 - val_loss: 0.4149 - val_accuracy: 0.9800

In [20]:
from tensorflow.keras.models import load_model

test_model = load_model(MODEL_PATH)
test_loss, test_acc = test_model.evaluate(test_data)

print(f"Test accuracy: {test_acc}")

63/63 [==============================] - 7s 111ms/step - loss: 0.5339 - accuracy: 0.9770
Test accuracy: 0.9769999980926514
